# README

Для работы необходима библиотека "instantclient_21_3" скачать ее можно с официального сайта Oracle (с помощью впн)

Известная проблема - oracle закрывает свое соедниение очень быстро, чтобы заново открыть соединение необходимо переоткрыть соеднинения

Еще проблема - очень долгое выполнение команды update например обновление одной записи для пустой таблицы терминалов занимает 79,5 ms, а поиск всех записей в терминале 79.8 ms. 
> %time v.execute(f"select * from {TERMINALS}")

> %time v.execute(f"""update {TERMINALS} set end_dt = to_date('{transaction.trans_date}', 'YYYY.MM.DD HH24:MI:SS')""")

А скорость работы поиска подозрительный действий при подборе сумм занимает с индексацией 91,3 ms без инедксации 214.

Вероятнее всего это сязано с тем, что для каждого запроса происходит переоткрытие соединение с нуля. Скорость должна возрасти если как-то объеденить серии запросов в один большой запрос.

В пунктах 2-4 происходит удаление сущетсвующих таблиц, заполнение новых, тестирование, удаление тестовых данных.
Для единичной прогонки для новой таблицы необходимо выполнить пункты 5-8. В пункте 9 закрываем открытые соединения. Пункт 10 содержит замер скорости для получения подбора сумм. 

In [110]:
%time v.execute(f"select * from {TERMINALS}")
%time v.execute(f"""update {TERMINALS} set end_dt = to_date('{transaction.trans_date}', 'YYYY.MM.DD HH24:MI:SS')""")

Wall time: 90.6 ms
Wall time: 90.4 ms


# Начало

In [1]:
import cx_Oracle
import pandas as pd

In [2]:
#у меня по такому пути находятся файлы, чтобы oracle был доступен из питона.
cx_Oracle.init_oracle_client(lib_dir= (input("Введити путь до библиотеки instantclient_21_3")))

In [ ]:
login = (input("введите логин a = "))
password = (input("введите пароль = "))

In [4]:
conn = cx_Oracle.connect(f"{login}/{password}@51.141.108.8:1521/dereboot")
c = conn.cursor()
v = conn.cursor()
c.execute(r'''select * from dual''')
for row in c:
    print(row)

('X',)


# СОЗДАЕМ ТАБЛИЦЫ

логика такая:

сначала инициализируем переменные с именами таблицы

потом удаляем существующие таблицы

потом создаем таблицы и нужную для работы программу логику:

1) создаем таблицу

2) создаем класс для представления в питоне, что по сути является частью промежуточной области.

3) определяем функцию, которая переводит данные из представления питона в хранилище данных

4) произвожу тестирование, перед основной работой программы внесенные данные почищу

In [5]:
CLIENTS = 'clients12'
ACCOUNTS = 'accounts21'
CARDS = 'cards12'
TERMINALS = 'terminals12'
TRANSACTION = 'transactions12'
META = 'meta12'

## drop tables. 

In [6]:
c.execute(f'drop table {CLIENTS}')

In [7]:
c.execute(f'drop table {ACCOUNTS}')

In [8]:
c.execute(f'drop table {CARDS}')

In [9]:
c.execute(f'drop table {TERMINALS}')

In [10]:
c.execute(f'drop table {TRANSACTION}')

In [11]:
c.execute('commit')

In [12]:
# считываем данные для проверки, потом - удалим
data = pd.read_excel("passport.xlsx")
data.shape
stage = data

## TRANSACTION

In [13]:
#create TRANSACTION
c.execute(f"""
CREATE TABLE {TRANSACTION}
    (
     trans_id               varchar (10)        NULL,       -- 1      --int64 (8 знаков)
     trans_date             DATE                NULL,       -- 2      --datetime64[ns] yyyy-mm-dd hh:mm:ss
     card_num               varchar (20 BYTE)   NULL,       -- 3      --object (20 знаков, числа )
     oper_type              varchar (40 BYTE)   NULL,       -- 14     --object максимум 11 кирилицей
     amt                    number (9,2)        NULL,       -- 15     --float64 (2 занка после запятой)
     oper_result            varchar (20 BYTE)   NULL,       -- 16     --object 7 знаков кирилицей
     terminal               varchar (10 BYTE)   NULL,       -- 17     --object 8 знаков 3латиница и 5 цифр
     start_dt               DATE                NULL,
     end_dt                 DATE                NULL
    )
""")

In [14]:
class class_transactions:
    def insert(self, start_dt):
        return f"""
insert into {TRANSACTION}(
   trans_id
  ,trans_date
  ,card_num
  ,oper_type
  ,amt
  ,oper_result
  ,terminal
  ,start_dt
)
values (
   '{self.trans_id}'
  ,to_date('{self.trans_date}', 'YYYY.MM.DD HH24:MI:SS')
  ,'{self.card_num}'
  ,'{self.oper_type}'
  , {self.amt}
  ,'{self.oper_result}'
  ,'{self.terminal}'
  ,to_date('{start_dt}', 'YYYY.MM.DD HH24:MI:SS')
)"""
    
    def __init__(self, trans_id, trans_date, card_num, oper_type, amt, oper_result, terminal):
        self.trans_id = trans_id
        self.trans_date = trans_date
        self.card_num = card_num        
        self.oper_type = oper_type
        self.amt = amt
        self.oper_result = oper_result
        self.terminal = terminal
        
    def __str__(self):
        return 'trans_id=' + str(self.trans_id) + ", trans_date=" + str(self.trans_date) + ", card_num=" + str(self.card_num) + ", oper_type=" + str(self.oper_type) + ', amt=' + str(self.amt) + ', oper_result=' + str(self.oper_result) + ', terminal=' + str(self.terminal)


In [15]:
def fill_transaction(new_data, index):
    return class_transactions(new_data.iloc[index][0],new_data.iloc[index][1],new_data.iloc[index][2],
                        new_data.iloc[index][13],new_data.iloc[index][14],new_data.iloc[index][15],new_data.iloc[index][16])

In [16]:
q = fill_transaction(data, 0)

In [17]:
c.execute(q.insert(q.trans_date))

In [18]:
c.execute(f"select * from {TRANSACTION}")
for row in c:
    print(row)

('1', datetime.datetime(2020, 5, 1, 0, 0, 29), '59649132026167121328', 'Пополнение', 31576.6, 'Отказ', 'POS43792', datetime.datetime(2020, 5, 1, 0, 0, 29), None)


## TERMINALS

In [19]:
#create TERMINALS
c.execute(f"""
CREATE TABLE {TERMINALS}
    (
     terminal               varchar (10 BYTE)   NULL,       -- 17           --object 8 знаков 3латиница и 5 цифр
     terminal_type          varchar (3 BYTE)    NULL,       -- 18           --object (3 знака латиница) POS/ATM
     terminal_city          varchar (50 BYTE)   NULL,       -- 19           --object max 22 кирилицей
     terminal_address       varchar (100 BYTE)  NULL,       -- 20           --object max 45 кирилицей
     start_dt               date     NULL,
     end_dt                 date NULL
    )
""")

In [20]:
class class_terminals:
    def insert(self, start_dt):
        return f"""
insert into {TERMINALS} (
  terminal
  ,terminal_type
  ,terminal_city
  ,terminal_address
  ,start_dt
)
values (
   '{self.terminal_id}'
  ,'{self.terminal_type}'
  ,'{self.terminal_city}'
  ,'{self.terminal_address}'
  ,to_date('{start_dt}', 'YYYY.MM.DD HH24:MI:SS')
)"""
    def __eq__(self, other):
        return self.terminal_id == other.terminal_id and self.terminal_type == other.terminal_type and self.terminal_city == other.terminal_city and self.terminal_address == other.terminal_address
       
    def __init__(self, terminal_id, terminal_type, terminal_city, terminal_address):
        self.terminal_id = terminal_id
        self.terminal_type = terminal_type
        self.terminal_city = terminal_city
        self.terminal_address = terminal_address
    def __str__(self):
        return "terminal_id=" + str(self.terminal_id) + ', terminal_type=' + str(self.terminal_type) +', terminal_city='+ str(self.terminal_city) + ', terminal_address=' + str(self.terminal_address)

In [21]:
def fill_terminal(new_data, index):
    return class_terminals(new_data.iloc[index][16], new_data.iloc[index][17],new_data.iloc[index][18],new_data.iloc[index][19])

In [22]:
w = fill_terminal(data, 0)
print(w)

terminal_id=POS43792, terminal_type=POS, terminal_city=Славянск-на-Кубани, terminal_address=Славянск-на-Кубани, ул. Клецкая, д. 86


In [23]:
c.execute(w.insert(q.trans_date))

In [24]:
c.execute(f"select * from {TERMINALS}")
for row in c:
    print(row)

('POS43792', 'POS', 'Славянск-на-Кубани', 'Славянск-на-Кубани, ул. Клецкая, д. 86', datetime.datetime(2020, 5, 1, 0, 0, 29), None)


## CARDS

In [25]:
#create CARDS
c.execute(f"""
CREATE TABLE {CARDS}
    (
     card_num               varchar (20 BYTE)   NOT NULL,   -- 3            --object (20 знаков, числа )
     account_num            varchar (20 BYTE)   NULL,       -- 4            --object (20 знаков, числа)
     start_dt               DATE                NOT NULL,
     end_dt                 DATE                NULL
    )
""")

In [26]:
class class_cards:
    def insert(self, start_dt):
        return f"""
insert into {CARDS} (
   card_num
  ,account_num
  ,start_dt
)
values (
   '{self.card_num}'
  ,'{self.account_num}'
  ,to_date('{start_dt}', 'YYYY.MM.DD HH24:MI:SS')
)"""
    def __eq__(self, other):
        return self.card_num == other.card_num and self.account_num == other.account_num
    
    def __init__(self, card_num, account_num):
        self.card_num = card_num
        self.account_num = account_num
    def __str__(self):
        return 'card_num=' + str(self.card_num) + ', account_num=' + str(self.account_num)

In [27]:
def fill_card(new_data, index):
    return class_cards(new_data.iloc[index][2], new_data.iloc[index][3])

In [28]:
e = fill_card(data, 0)
print(e)

card_num=59649132026167121328, account_num=40817810000001139973


In [29]:
c.execute(e.insert(q.trans_date))

In [30]:
c.execute(f"select * from {CARDS}")
for row in c:
    print(row)

('59649132026167121328', '40817810000001139973', datetime.datetime(2020, 5, 1, 0, 0, 29), None)


## ACCOUNTS

In [31]:
#create ACCOUNTS
c.execute(f"""
CREATE TABLE {ACCOUNTS}
    (
     account_num            varchar (20 BYTE)   NOT NULL,   -- 4            --object (20 знаков, числа)
     valid_to               date                NULL,       -- 5            --date yyyy-mm-dd
     client                 varchar (7 BYTE)    NULL,       -- 6            --object (7знаков, числа-)
     start_dt               DATE                NOT NULL,
     end_dt                 DATE                NULL
    )
""")

In [32]:
class class_accounts:
    def insert(self, start_dt):
        return f"""
insert into {ACCOUNTS} (
   account_num
  ,valid_to
  ,client 
  ,start_dt
)
values (
  '{self.account_num}'
  ,to_date('{self.valid_to}','YYYY.MM.DD HH24:MI:SS')
  ,'{self.client}'
  ,to_date('{start_dt}','YYYY.MM.DD HH24:MI:SS')
)"""
    def __eq__(self, other):
        return self.account_num == other.account_num and self.valid_to == other.valid_to and self.client == other.client
    
    def __init__(self, account_num, valid_to, client):
        self.account_num = account_num
        self.valid_to = valid_to
        self.client = client
        
    def __str__(self):
        return 'account_num=' + str(self.account_num) + ', valid_to=' + str(self.valid_to) + ', client=' + str(self.client)


In [33]:
def fill_account(new_data, index):
    return class_accounts(new_data.iloc[index][3],new_data.iloc[index][4],new_data.iloc[index][5])

In [34]:
r = fill_account(data, 0)
print(r)

account_num=40817810000001139973, valid_to=2036-01-16 00:00:00, client=3-95179


In [35]:
c.execute(r.insert(q.trans_date))

In [36]:
c.execute(f"select * from {ACCOUNTS}")
for row in c:
    print(row)

('40817810000001139973', datetime.datetime(2036, 1, 16, 0, 0), '3-95179', datetime.datetime(2020, 5, 1, 0, 0, 29), None)


## CLIENTS

In [37]:
#create CLIENTS
c.execute(f"""
CREATE TABLE {CLIENTS}
    (
     client_id              varchar (7 BYTE)    NOT NULL,   -- 6            --object (7знаков, числа-)
     last_name              varchar (40 BYTE)   NULL,       -- 7            --object кирилица
     first_name             varchar (40 BYTE)   NULL,       -- 8            --object кирилица   
     patronymic             varchar (40 BYTE)   NULL,       -- 9            --object кирилица
     date_of_birth          date                NULL,       -- 10           --date yyyy-mm-dd
     passport_num           varchar (10 BYTE)   NULL,       -- 11           --int64 10 знаков
     passport_valid_to      date                NULL,       -- 12           --date yyyy-mm-dd
     phone                  varchar (11 BYTE)   NULL,       -- 13           --int64 11знаков
     start_dt               date                NOT NULL,
     end_dt                 date                NULL
    )
""")

In [38]:
class class_clients:
    def insert(self, start_dt):
        return f"""
insert into {CLIENTS} (
  client_id
  ,last_name
  , first_name
  , patronymic
  , date_of_birth
  , passport_num
  , passport_valid_to
  , phone
  , start_dt
)
values (
  '{self.client_id}'
  , '{self.last_name}'
  , '{self.first_name}'
  , '{self.patrinmic}'
  , to_date('{self.date_of_birth}','YYYY.MM.DD HH24:MI:SS')
  , '{self.passport_num}'
  , to_date('{self.passport_valid_to}','YYYY.MM.DD HH24:MI:SS') 
  , '{self.phone}'
  , to_date('{start_dt}','YYYY.MM.DD HH24:MI:SS')
)"""
    
    def __eq__(self, other):
        return self.client_id == other.client_id and    self.last_name == other.last_name and     self.first_name == other.first_name and    self.patrinmic == other.patrinmic and    self.date_of_birth == other.date_of_birth and    str(self.passport_num) == str(other.passport_num) and    self.passport_valid_to == other.passport_valid_to and    str(self.phone) == str(other.phone)
    
    def __init__(self, client_id, last_name, first_name, patrinmic, date_of_birth, passport_num, passport_valid_to, phone):
        self.client_id = client_id
        self.last_name = last_name
        self.first_name = first_name
        self.patrinmic = patrinmic
        self.date_of_birth = date_of_birth
        self.passport_num = passport_num
        self.passport_valid_to = passport_valid_to
        self.phone = phone
        
    def __str__(self):
        return 'client_id='+str(self.client_id) + ', last_name=' + str(self.last_name) + ', first_name=' + str(self.first_name)+    ', patrinmic=' + str(self.patrinmic) + ', date_of_birth=' + str(self.date_of_birth) + ', passport_num=' + str(self.passport_num)+    ', passport_valid_to=' + str(self.passport_valid_to) + ', phone=' + str(self.phone)

In [39]:
def fill_client(new_data, index):
    return class_clients(new_data.iloc[index][5],new_data.iloc[index][6],new_data.iloc[index][7],new_data.iloc[index][8],new_data.iloc[index][9],
                  new_data.iloc[index][10],new_data.iloc[index][11],new_data.iloc[index][12])

In [40]:
t = fill_client(data, 0)
print(t)

client_id=3-95179, last_name=Мисик, first_name=Сергей, patrinmic=Николаевич, date_of_birth=1938-06-25 00:00:00, passport_num=7076445954, passport_valid_to=2019-11-09 00:00:00, phone=79497481039


In [41]:
c.execute(t.insert(q.trans_date))

In [42]:
c.execute(f"select * from {CLIENTS}")
for row in c:
    print(row)

('3-95179', 'Мисик', 'Сергей', 'Николаевич', datetime.datetime(1938, 6, 25, 0, 0), '7076445954', datetime.datetime(2019, 11, 9, 0, 0), '79497481039', datetime.datetime(2020, 5, 1, 0, 0, 29), None)


## Создаем индексы

In [43]:
# create index QWE on terminals1(terminal, end_dt) - попробовать
c.execute(f"create index IX_{TRANSACTION}_TRANS_ID on {TRANSACTION}(trans_id)")
c.execute(f"create index IX_{TERMINALS}_TERMINAL_END_DT on {TERMINALS}(terminal, end_dt)")
c.execute(f"create index IX_{CARDS}_card_num_end_dt on {CARDS}(card_num, end_dt)")
c.execute(f"create index IX_{ACCOUNTS}_account_num_end_dt on {ACCOUNTS}(account_num, end_dt)")
c.execute(f"create index IX_{CLIENTS}_client_id_end_dt on {CLIENTS}(client_id, end_dt)")
c.execute('commit')

## Создаем ограничения

не могу понять, как надо делать primaryKey на transactions, чтобы можно было сослаться на него в terminals и cards.
Есть вариант навесить его на (card_num, terminal_id, st_dt)
В terminals добавить card_num, а в cards -  terminal_id и нормально ссылать на ключ, но какое-то решение не очень на мой взгляд

## mata-info

с метой слегка другая логика - в ней я питоновское представление достаю из sql-таблицы.

In [44]:
conn = cx_Oracle.connect(f"{login}/{password}@51.141.108.8:1521/dereboot")
c = conn.cursor()
v = conn.cursor()
c.execute(r'''select * from dual''')
for row in c:
    print(row)

('X',)


In [45]:
c.execute(f'drop table {META}')

In [46]:
#create meta-info
c.execute(f"""
CREATE TABLE {META}
    (
     last_update_date   DATE   NULL,       
     passport_date      DATE   NULL,    
     account_date       DATE   NULL,       
     city_date          DATE   NULL, 
     sum_date           DATE   NULL,       
     start_dt           DATE   NULL,
     end_dt             DATE   NULL
    )
""")

вношу данные в таблицу

In [47]:
c.execute(f"""
insert into {META} (last_update_date, passport_date, account_date, city_date, sum_date, start_dt)
values (
    to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS')
)
""")

In [48]:
c.execute('commit')

In [49]:
class meta:
    def __init__(self, a,b,c,d,e):
        self.last_update_date = pd.Timestamp(a)
        self.passport_date = pd.Timestamp(b)
        self.account_date = pd.Timestamp(c)
        self.city_date = pd.Timestamp(d)
        self.sum_date = pd.Timestamp(e)

In [50]:
c.execute(f"select * from {META} where end_dt is null")

<cx_Oracle.Cursor on <cx_Oracle.Connection to student_ivashin@51.141.108.8:1521/dereboot>>

In [51]:
for row in c:
    print(row)
    meta_instance = meta(row[0],row[1],row[2],row[3],row[4])

(datetime.datetime(1990, 1, 1, 0, 0), datetime.datetime(1990, 1, 1, 0, 0), datetime.datetime(1990, 1, 1, 0, 0), datetime.datetime(1990, 1, 1, 0, 0), datetime.datetime(1990, 1, 1, 0, 0), datetime.datetime(1990, 1, 1, 0, 0), None)


проверка

In [52]:
meta_instance.last_update_date

Timestamp('1990-01-01 00:00:00')

# чищу данные после тестирования

In [53]:
def test_case():
    """Подчищаем все данные из таблица, нужно для отладки программы"""
    c.execute(f'delete from {TERMINALS} where 1=1')
    c.execute(f'delete from {CARDS} where 1=1')
    c.execute(f'delete from {TRANSACTION} where 1=1')
    c.execute(f'delete from {ACCOUNTS} where 1=1')
    c.execute(f'delete from {CLIENTS} where 1=1')
    c.execute(f'delete from {META} where 1=1')
    c.execute(f"""
    insert into {META} (last_update_date, passport_date, account_date, city_date, sum_date, start_dt)
    values (
        to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
        to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
        to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
        to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
        to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS'),
        to_date('1990.01.01 00:00:00', 'YYYY.MM.DD HH24:MI:SS')
    )
    """)
    c.execute('commit')

In [54]:
test_case()

# Чтение данных

In [85]:
data_path = (input("введите путь к таблице "))

введите путь к таблице pays_2.xlsx


In [86]:
data = pd.read_excel(f"{data_path}")
stage = pd.DataFrame(columns = data.columns)

In [87]:
for i in range(len(data)):
    #считали построчно из источника данные в stage
    if(data.loc[i].date > meta_instance.last_update_date):
        stage.loc[i] = data.loc[i]

# Основная работа

In [88]:
conn = cx_Oracle.connect(f"{login}/{password}@51.141.108.8:1521/dereboot")
c = conn.cursor()
v = conn.cursor()
c.execute(r'''select * from dual''')
for row in c:
    print(row)

('X',)


In [89]:
counter = 0
for i in range(len(stage)):
    # вероятнее всего надо сделать проверку, что stage.date > meta.date
    
    #вносим транзакцию
    transaction = fill_transaction(stage, i)
    print(transaction)
    #если дата транакции <= даты обновления мета-информации, то такую траназкцию пропускаем
    if (transaction.trans_date <= meta_instance.last_update_date):
        continue
    c.execute(transaction.insert(transaction.trans_date))
    
    
#   вносим терминал
    terminal = fill_terminal(stage, i)
    c.execute(f"select count(*) from {TERMINALS} where terminal='{terminal.terminal_id}'")
    for row in c:
        counter = row[0]
        
    if (counter == 0):
        c.execute(terminal.insert(transaction.trans_date))
    else :
        c.execute(f"select * from {TERMINALS} WHERE terminal='{terminal.terminal_id}' and end_dt is null")
        #тут будет одна сктрока - для которой отсутвует end_dt
        for row in c:
#             print(row)
            
            terminal_from_sql = class_terminals(row[0], row[1], row[2], row[3])
            if(terminal_from_sql != terminal):
                print("not equal")
                v.execute(f"""update {TERMINALS} set end_dt = to_date('{transaction.trans_date}', 'YYYY.MM.DD HH24:MI:SS') 
                          where terminal='{terminal.terminal_id}' and end_dt is null""")
                v.execute(terminal.insert(transaction.trans_date))
            
    #вносим карты
    card = fill_card(stage, i)
    c.execute(f"select count(*) from {CARDS} where card_num='{card.card_num}'")
    for row in c:
        counter = row[0]
    #сделал аналогично транзакциям
    if (counter == 0):
        c.execute(card.insert(transaction.trans_date))
    else :
        print(f"повторная карта {card.card_num}")
        c.execute(f"select * from {CARDS} WHERE card_num='{card.card_num}' AND end_dt is null")
        for row in c:
            card_from_sql = class_cards(row[0], row[1])
            if(card_from_sql != card):
                v.execute(f"""update {CARDS} set end_dt = to_date('{transaction.trans_date}', 'YYYY.MM.DD HH24:MI:SS') 
                          where card_num='{card.card_num}' and end_dt is null""")
                v.execute(card.insert(transaction.trans_date))
        
    #вносим аккаунты
    account = fill_account(stage, i)
    c.execute(f"select count(*) from {ACCOUNTS} where account_num='{account.account_num}'")
    for row in c:
        counter = row[0]
    if (counter == 0):
        c.execute(account.insert(transaction.trans_date))
    else :
        c.execute(f"select * from {ACCOUNTS} WHERE account_num='{account.account_num}' AND end_dt is null")
#         #тут будет одна сктрока - для которой отсутвует end_dt
        for row in c:
            account_from_sql = class_accounts(row[0], row[1], row[2])
            if(account_from_sql != account):
#                 print(account.account_num)
                v.execute(f"""update {ACCOUNTS} set end_dt = to_date('{transaction.trans_date}', 'YYYY.MM.DD HH24:MI:SS') 
                          where account_num='{account.account_num}' and end_dt is null""")
                v.execute(account.insert(transaction.trans_date))

    #вносим клиентов
    client = fill_client(stage, i)
    c.execute(f"select count(*) from {CLIENTS} where client_id='{client.client_id}'")
    for row in c:
        counter = row[0]
    if (counter == 0):
        c.execute(client.insert(transaction.trans_date))
    else:
        c.execute(f"select * from {CLIENTS} WHERE client_id='{client.client_id}' AND end_dt is null")
        for row in c:
            client_from_sql = class_clients(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7])
            if(client_from_sql != client):
                v.execute(f"""update {CLIENTS} set end_dt = to_date('{transaction.trans_date}', 'YYYY.MM.DD HH24:MI:SS') 
                          where client_id='{client.client_id}' and end_dt is null""")
                v.execute(client.insert(transaction.trans_date))
                
    c.execute("commit")
    v.execute('commit')

trans_id=33, trans_date=2020-05-02 00:05:00, card_num=42810999157069197448, oper_type=Оплата, amt=1, oper_result=Успешно, terminal=POS64475
повторная карта 42810999157069197448
trans_id=36, trans_date=2020-05-02 00:05:00, card_num=43440125275047901246, oper_type=Оплата, amt=10, oper_result=Отказ, terminal=POS20393
повторная карта 43440125275047901246
trans_id=37, trans_date=2020-05-02 00:10:00, card_num=43440125275047901246, oper_type=Оплата, amt=1, oper_result=Успешно, terminal=POS20393
повторная карта 43440125275047901246
trans_id=38, trans_date=2020-05-02 00:00:00, card_num=56394328398014656631, oper_type=Пополнение, amt=1000, oper_result=Отказ, terminal=ATM45335
trans_id=39, trans_date=2020-05-02 00:05:00, card_num=56394328398014656631, oper_type=Пополнение, amt=100, oper_result=Отказ, terminal=ATM45335
повторная карта 56394328398014656631
trans_id=40, trans_date=2020-05-02 00:10:00, card_num=56394328398014656631, oper_type=Пополнение, amt=10, oper_result=Отказ, terminal=ATM45335
п

In [90]:
meta_instance.last_update_date = stage['date'].max()

# Построение отчета

## определяем вспомагательные функции

Логика следующая:
1) определяем две функции для корректной состыковки таблиц на основе start_dt и end_dt
2) определяем функцию, которая красиво будет переписывать строку из sql-запросов в .txt файл

In [91]:
def getter_cards_accounts_clients():
    '''
возвращает условие для корректного джоина
    '''
    return f"""
{TRANSACTION}.trans_date >= {CARDS}.start_dt and {TRANSACTION}.trans_date < coalesce({CARDS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD')) 
and
{TRANSACTION}.trans_date >= {ACCOUNTS}.start_dt and {TRANSACTION}.trans_date < coalesce({ACCOUNTS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD')) 
and
{TRANSACTION}.trans_date >= {CLIENTS}.start_dt and {TRANSACTION}.trans_date < coalesce({CLIENTS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD'))     
    """

In [92]:
def getter_cards_accounts_clients_terminals():
    '''
возвращает условие для корректного джоина
    '''
    return f"""
{TRANSACTION}.trans_date >= {CARDS}.start_dt and {TRANSACTION}.trans_date < coalesce({CARDS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD')) 
and
{TRANSACTION}.trans_date >= {ACCOUNTS}.start_dt and {TRANSACTION}.trans_date < coalesce({ACCOUNTS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD')) 
and
{TRANSACTION}.trans_date >= {CLIENTS}.start_dt and {TRANSACTION}.trans_date < coalesce({CLIENTS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD'))  
and
{TRANSACTION}.trans_date >= {TERMINALS}.start_dt and {TRANSACTION}.trans_date < coalesce({TERMINALS}.end_dt,to_date('9999:12:12', 'YYYY:MM:DD'))     
    """

In [93]:
def row_to_fraud_str(row):
    """преобразуем полученную строку в строку для отчета"""
    return (f"дата транзакции '{row[0]}'" +
            f", номер паспорта клиента '{row[1]}'" +
            f", ФИО клиента '{row[2]} {row[3]} {row[4]}'" +
            f", номер телефона '{row[5]}'" +
            f", описание типа предпалагаемого мошеничества '{row[6]}'" +
            f", время построения отчета '{row[7]}'" +
            "\n")

In [94]:
f = open('text.txt', 'a')

## Выполняем поиск нарушений

Логика следующая:
1) выполняем select-запрос, который находит все нарушения по типу

2) записываем полученные результаты в .txt файл и выводим в output (для наглядности)

3) обновляем мета-информацию:

Для нарушений по паспорту и договору meta информация содержит максимальную дату из stage.

Для разных городов и подбора сумм логика обновления соответсвующей meta-информации немножного другая. Тк нам необходимо учитывать для функции LAG данные с прошлого дня. Поэтому в meta информацию мы кладем 23:40 или 23:00. Старые данные не будут учитываться дважды, тк в конечном select мы делаем условие вывода данные только с новой даты с помощью функции get_date(meta_insatnce.last_update_date) - она вернет полночь наступившей даты, тк мета информация уже была обновлена.

### просроченный паспорт

In [95]:
c.execute(f"""
select 
    fraud_dt
    , passport
    , last_name
    , first_name
    , patronymic
    , phone
    , fraud_type
    , report_dt
from (
    select 
        trans_date as fraud_dt,
        passport_num as passport,
        last_name,
        first_name,
        patronymic,
        phone,
        'passport is outdated' as fraud_type,
        sysdate as report_dt,
        trans_date 
        , passport_valid_to
        , passport_valid_to - trans_date
        , case 
            when passport_valid_to + INTERVAL '1' DAY - trans_date > 0 then 0 --добавляем 1 день к дате паспорта
            else 1
        end as flag
    from {TRANSACTION}
        left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
        left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
        left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
    where trans_date > to_date('{meta_instance.passport_date}', 'YYYY.MM.DD HH24:MI:SS') 
    and {getter_cards_accounts_clients()}
)
where flag = 1
""")

<cx_Oracle.Cursor on <cx_Oracle.Connection to student_ivashin@51.141.108.8:1521/dereboot>>

In [96]:
for row in c:
    f.write(row_to_fraud_str(row))
    print(row)
f.flush()

In [97]:
#обновляем информацию меты. В качестве максимальной даты берем время самого последнего запроса из stage.
meta_instance.passport_date = meta_instance.last_update_date
meta_instance.passport_date

Timestamp('2020-05-02 00:15:00')

### просроченный договор

In [98]:
c.execute(f"""
select 
    fraud_dt
    , passport
    , last_name
    , first_name
    , patronymic
    , phone
    , fraud_type
    , report_dt
from (
    select 
        trans_date as fraud_dt,
        passport_num as passport,
        last_name,
        first_name,
        patronymic,
        phone,
        'account is outdated' as fraud_type,
        sysdate as report_dt,
        trans_date 
        , passport_valid_to
        , valid_to - trans_date
        , case 
            when valid_to + INTERVAL '1' DAY - trans_date > 0 then 0
            else 1
        end as flag
    from {TRANSACTION}
    left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
    left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
    left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
    where trans_date > to_date('{meta_instance.account_date}', 'YYYY.MM.DD HH24:MI:SS') 
    and {getter_cards_accounts_clients()}
)
where flag = 1
""")

<cx_Oracle.Cursor on <cx_Oracle.Connection to student_ivashin@51.141.108.8:1521/dereboot>>

In [99]:
print(f"{meta_instance.account_date}")

2020-05-01 23:55:00


In [100]:
for row in c:
    f.write(row_to_fraud_str(row))
    print(row)
f.flush()

In [101]:
#обновляем информацию меты
meta_instance.account_date = meta_instance.last_update_date
meta_instance.account_date

Timestamp('2020-05-02 00:15:00')

### Разные города

In [102]:
def get_date(q):
    """получаем полночь переданной даты"""
    return str(q.year) + '.' + str(q.month) + '.' + str(q.day) + ' 00:00:00'

In [103]:
c.execute(f"""
select 
    fraud_dt,
    passport,
    last_name,
    first_name,
    patronymic,
    phone,
    'city ?' as fraud_type,
    sysdate as report_dt
from 
(    
    select 
            t.* 
    from 
    (
        select 
            trans_id,
            trans_date
            , {ACCOUNTS}.client as client
            , to_char (trans_date, 'YYYY.MM.DD HH24:MI:SS') as dt
            , to_char (LAG (trans_date , 1 , to_date('1900.01.01 10:10:10', 'YYYY.MM.DD HH24:MI:SS')) OVER (PARTITION by client order by trans_date), 'YYYY.MM.DD HH24:MI:SS')  as lag_dt
            , round((trans_date - LAG (trans_date , 1 , to_date('1900.01.01 10:10:10', 'YYYY.MM.DD HH24:MI:SS')) OVER (PARTITION by client order by trans_date) ) * 24 * 60, 2) as diff --разница в минутах
            , terminal_city
            , LAG (terminal_city, 1, null) OVER (PARTITION by client order by trans_date) as city2
            , trans_date as fraud_dt
            , passport_num as passport
            , last_name
            , first_name
            , patronymic
            , phone
        from {TRANSACTION}
            left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
            left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
            left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
            left join {TERMINALS} on {TERMINALS}.terminal = {TRANSACTION}.terminal
        where trans_date > to_date('{meta_instance.city_date}', 'YYYY.MM.DD HH24:MI:SS')
        and {getter_cards_accounts_clients_terminals()}
        order by client_id, trans_date
    ) t
    where diff <= 60 and terminal_city != city2 and trans_date > to_date('{get_date(meta_instance.last_update_date)}', 'YYYY.MM.DD HH24:MI:SS')
) ret""")

<cx_Oracle.Cursor on <cx_Oracle.Connection to student_ivashin@51.141.108.8:1521/dereboot>>

In [104]:
def date_minus_hour(q):
    """Получаем датадайм для даты и времени 23:00:00"""
    return str(q.year) + '.' + str(q.month) + '.' + str(q.day) + ' 23:00:00'

In [105]:
i = 0
for row in c:
    i = i + 1
    f.write(row_to_fraud_str(row))
    print(row)
f.flush()
print(i)

0


In [106]:
#обновляем мета-информацию. В качестве максимальной даты берем время самого последнего запроса из stage.
c.execute(f'select max(trans_date) from {TRANSACTION}')
#снова та же проблема, не знаю как указать ронво 1 строку, поэтому делаю так
for row in c:
    meta_instance.city_date = date_minus_hour(row[0])

### Попытка подбора сумм

In [107]:
c.execute(f"""
select 
    trans_date as fraud_dt,
    passport,
    last_name,
    first_name,
    patronymic,
    phone,
    'summ ?' as fraud_type,
    sysdate as report_dt
from (
    select 
        trans_date,
        oper_result,
        trans_id,
        clt,
        amt1,
        amt2,
        amt3,
        amt4,
        to_char(dt1, 'YYYY.MM.DD HH24:MI:SS') dt1,
        to_char(dt2, 'YYYY.MM.DD HH24:MI:SS') dt2,
        to_char(dt3, 'YYYY.MM.DD HH24:MI:SS') dt3,
        to_char(dt3, 'YYYY.MM.DD HH24:MI:SS') dt4,
        round((dt1 - dt4) * 24 * 60, 2) diff_min,
        passport,
        last_name,
        first_name,
        patronymic,
        phone
    from(
        select 
            trans_date,
            trans_id,
            client_id as clt,
            oper_result,
            amt as amt1,
            LAG(AMT, 1, null) OVER (PARTITION by client order by trans_date) as amt2,
            LAG(AMT, 2, null) OVER (PARTITION by client order by trans_date) as amt3,
            LAG(AMT, 3, null) OVER (PARTITION by client order by trans_date) as amt4,
            trans_date as dt1,
            LAG(trans_date, 1, null) OVER (PARTITION by client order by trans_date) as dt2,
            LAG(trans_date, 2, null) OVER (PARTITION by client order by trans_date) as dt3,
            LAG(trans_date, 3, null) OVER (PARTITION by client order by trans_date) as dt4,
            client_id,
            passport_num as passport,
            last_name,
            first_name,
            patronymic,
            phone
        from 
        (
            select 
                *
            from {TRANSACTION}
                left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
                left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
                left join {CLIENTS} on {ACCOUNTS}.client = {CLIENTS}.client_id
                left join {TERMINALS} on {TERMINALS}.terminal = {TRANSACTION}.terminal
                where trans_date > to_date('{meta_instance.sum_date}', 'YYYY.MM.DD HH24:MI:SS')
                and {getter_cards_accounts_clients_terminals()}
            order by client_id, trans_date
        ) t
    ) tmp
) res
where diff_min <= 20 and amt1 < amt2 and amt2 < amt3 and amt3 < amt4 and oper_result='Успешно' and trans_date > to_date('{get_date(meta_instance.last_update_date)}', 'YYYY.MM.DD HH24:MI:SS')
""")

<cx_Oracle.Cursor on <cx_Oracle.Connection to student_ivashin@51.141.108.8:1521/dereboot>>

In [108]:
def date_minus_20_minutes(q):
    """Получаем датадайм для даты и времени 23:40:00"""
    return str(q.year) + '.' + str(q.month) + '.' + str(q.day) + ' 23:40:00'

In [109]:
i = 0
for row in c:
    i = i + 1
    f.write(row_to_fraud_str(row))
    print(row)
f.flush()
print(i)

(datetime.datetime(2020, 5, 2, 0, 15), '1298690726', 'Омелюшкин', 'Василий', 'Иванович', '79736583722', 'summ ?', datetime.datetime(2022, 5, 10, 11, 10, 20))
(datetime.datetime(2020, 5, 2, 0, 10), '7173036990', 'Поджарый', 'Роман', 'Александрович', '79343500689', 'summ ?', datetime.datetime(2022, 5, 10, 11, 10, 20))
(datetime.datetime(2020, 5, 2, 0, 5), '3602724796', 'Сноркин', 'Николай', 'Антонович', '79890926038', 'summ ?', datetime.datetime(2022, 5, 10, 11, 10, 20))
3


In [81]:
#обновляем мета-информацию
c.execute(f'select max(trans_date) from {TRANSACTION}')
#снова та же проблема, не знаю как указать ронво 1 строку, поэтому делаю так
for row in c:
    meta_instance.sum_date = date_minus_20_minutes(row[0])

# Вносим мету в бд

In [82]:
c.execute(f"""update {META} set end_dt = to_date('{meta_instance.last_update_date}', 'YYYY.MM.DD HH24:MI:SS') 
                          where  end_dt is null""")

In [83]:
c.execute(f"""
insert into {META} (last_update_date, passport_date, account_date, city_date, sum_date, start_dt)
values (
    to_date('{meta_instance.last_update_date}', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('{meta_instance.passport_date}', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('{meta_instance.account_date}', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('{meta_instance.city_date}', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('{meta_instance.sum_date}', 'YYYY.MM.DD HH24:MI:SS'),
    to_date('{meta_instance.last_update_date}', 'YYYY.MM.DD HH24:MI:SS')
)
""")

In [84]:
c.execute('commit')

# Конец
Закрываем все соединения

In [356]:
f.close()
c.close()
v.close()

# замер скорости для подбора сумм

Я проводил замер скорости сразу для трех дней. Этого можно добиться если после основной работы алгоритма обновить мета-информацию, затем закрыть соединения (для чистоты эксперимента) и переоткрыть его заново.

In [352]:
conn = cx_Oracle.connect(f"{login}/{password}@51.141.108.8:1521/dereboot")
c = conn.cursor()
v = conn.cursor()
c.execute(r'''select * from dual''')
for row in c:
    print(row)

('X',)


In [353]:
def qwe():
    c.execute(f"""
    select 
        fraud_dt,
        passport,
        last_name,
        first_name,
        patronymic,
        phone,
        'city ?' as fraud_type,
        sysdate as report_dt
    from 
    (    
        select 
                t.* 
        from 
        (
            select 
                trans_id,
                trans_date
                , {ACCOUNTS}.client as client
                , to_char (trans_date, 'YYYY.MM.DD HH24:MI:SS') as dt
                , to_char (LAG (trans_date , 1 , to_date('1900.01.01 10:10:10', 'YYYY.MM.DD HH24:MI:SS')) OVER (PARTITION by client order by trans_date), 'YYYY.MM.DD HH24:MI:SS')  as lag_dt
                , round((trans_date - LAG (trans_date , 1 , to_date('1900.01.01 10:10:10', 'YYYY.MM.DD HH24:MI:SS')) OVER (PARTITION by client order by trans_date) ) * 24 * 60, 2) as diff --разница в минутах
                , terminal_city
                , LAG (terminal_city, 1, null) OVER (PARTITION by client order by trans_date) as city2
                , trans_date as fraud_dt
                , passport_num as passport
                , last_name
                , first_name
                , patronymic
                , phone
            from {TRANSACTION}
                left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
                left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
                left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
                left join {TERMINALS} on {TERMINALS}.terminal = {TRANSACTION}.terminal
            where trans_date > to_date('{meta_instance.city_date}', 'YYYY.MM.DD HH24:MI:SS')
            and {getter_cards_accounts_clients_terminals()}
            order by client_id, trans_date
        ) t
        where diff <= 60 and terminal_city != city2 and trans_date > to_date('{get_date(meta_instance.last_update_date)}', 'YYYY.MM.DD HH24:MI:SS')
    ) ret""")

In [354]:
%time qwe()

Wall time: 91.3 ms
